# my voki tester notebook

In [1]:
import datetime
import numpy as np
import pandas as pd
import os
from pathlib import Path
import pickle
import sys
from hashlib import md5
import pyvoki as vo
import random

## settings

In [2]:
# go to root
root = Path('C:/coding/voki/')
os.chdir(root)
assert os.getcwd().endswith('voki'), f'The cwd is not "voki" but {os.getcwd()}'

# make jupyter reload the package
%load_ext autoreload
%autoreload 2

# set the seed for reproducable results
np.random.seed(123)

In [3]:
N_LEVELS = 3
MAX_BOX_CARDS = 5

db_raw_fpath = root / 'data/voki_export_full_db.csv'
db_raw = pd.read_csv(db_raw_fpath,sep=';')
db_raw['book']=db_raw['file'].str.slice(start=31, stop=32).astype(int)
db_raw.head(1)
data_dir = root/ 'data'
trainer_dir = root / 'db/v1/'

In [4]:
box = vo.VokiBox(n=N_LEVELS, max_cards=MAX_BOX_CARDS)
trainer = vo.VokiTrainer(db_raw,
                          trainer_dir,
                          n_levels=N_LEVELS,
                          max_box_cards=MAX_BOX_CARDS,
                          voki_box=box)
trainer

Voki Trainer with 3637 entries in db

In [5]:
trainer.box.fill(trainer.get_next_box_cards())

In [6]:
trainer.show_box()

Voki Box with 5/5 cards
level 1
    eine Lektion
    eins
    männl. Vorname; hier: Name eines Papageis
    in
    Hauptstadt Frankreichs
level 2
level 3


In [7]:
trainer.do_vokibox()

"eine Lektion":  une lecon


wrong. => "une lecon" != "une leçon"


"eins":  1 = un


correct!


"männl. Vorname; hier: Name eines Papageis":  arthur


wrong. => "arthur" != "Arthur"


"in":  á


wrong. => "á" != "à"


"Hauptstadt Frankreichs":  Paris


correct!


In [8]:
trainer.show_box()

Voki Box with 5/5 cards
level 1
    eine Lektion
    männl. Vorname; hier: Name eines Papageis
    in
level 2
    eins
    Hauptstadt Frankreichs
level 3


In [9]:
trainer.do_vokibox()

"eins":  1 = un


correct!


"Hauptstadt Frankreichs":  Paris


correct!


"eine Lektion":  une leçon


correct!


"männl. Vorname; hier: Name eines Papageis":  Arthur


correct!


"in":  à


correct!


In [10]:
trainer.show_box()

Voki Box with 5/5 cards
level 1
level 2
    eine Lektion
    männl. Vorname; hier: Name eines Papageis
    in
level 3
    eins
    Hauptstadt Frankreichs


In [11]:
trainer.do_vokibox()

"eins":  un


wrong. => "un" != "1 = un"


"Hauptstadt Frankreichs":  Paris


correct!


"eine Lektion":  une leçon


correct!


"männl. Vorname; hier: Name eines Papageis":  Arthur


correct!


"in":  à


correct!


In [12]:
trainer.show_box()

Voki Box with 4/5 cards
level 1
    eins
level 2
level 3
    eine Lektion
    männl. Vorname; hier: Name eines Papageis
    in


In [13]:
trainer.do_vokibox()

"eine Lektion":  une leçon


correct!


"männl. Vorname; hier: Name eines Papageis":  Arthur


correct!


"in":  à


correct!


"eins":  1 = un


correct!


"Guten Tag!":  Bonjour!


correct!


In [14]:
trainer.show_box()

Voki Box with 2/5 cards
level 1
level 2
    eins
    Guten Tag!
level 3


In [15]:
trainer.do_vokibox()

2022-02-27 21:03:36.995 | WARNING  | pyvoki.main:check_card_already_in_box:343 - Card fc6f4b16c99f0cbcb3e6f18e169df236 is already in this box at level 2


"eins":  1 = un


correct!


"Guten Tag!":  Bonjour!


correct!


"eine Straße":  une ru


"une ru" != "une rue"


"das ist":  c'est


correct!


In [16]:
trainer.show_box()

Voki Box with 4/5 cards
level 1
    eine Straße
level 2
    das ist
level 3
    eins
    Guten Tag!


In [6]:
level = 4
hash_ = trainer.db.iloc[0].name
solution = trainer.db.loc[hash_]['text']
trainer.box.onboard_card(hash_, level=level)


True

In [8]:
trainer.attempt_card_in_level(lvl=level)

eine Lektion une leçon


correct!


True

In [26]:
b = trainer.box.get_all_cards()
b

['2ced79c45d1ccdbdd2e01726d87e1dae',
 '66932ff7f2834ad68c9a9dbfa393d3ec',
 'c0b716ccc9c4912796f42843354f270b',
 'fc6f4b16c99f0cbcb3e6f18e169df236']

In [24]:
df = trainer.db[trainer.db['attempts'].apply(len) < trainer.box.n_levels]

,text,text_g,text_s,y_max,page,file,book,attempts,streak,last_attempt_time,last_successful_attempt_time,passed_vokibox,box_shortlisted
hash,,,,,,,,,,,,,
fc6f4b16c99f0cbcb3e6f18e169df236,Bonjour!,Guten Tag!,"~, Arthur!",1171,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Successful attempt at 2022-02-27 21:02:47: ""B...",2,2022-02-27 21:04:41,2022-02-27 21:04:41,False,NaN
2ced79c45d1ccdbdd2e01726d87e1dae,une rue,eine Straße,nan,1192,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Failed attempt at 2022-02-27 21:04:47: ""une ru""]",0,2022-02-27 21:04:47,NaN,False,NaN
66932ff7f2834ad68c9a9dbfa393d3ec,c'est,das ist,nan,1216,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Successful attempt at 2022-02-27 21:04:51: ""c...",1,2022-02-27 21:04:51,2022-02-27 21:04:51,False,NaN
801dbee798c9ff4d905f7537f1fe0ded,monsieur ...,Herr ...,nan,1240,2,C:\coding\voki\data\SerieVertes1.pdf,1,[],0,NaN,NaN,False,NaN
ad158daf1773b311a1a61929c9bf019b,Rigot,Familienname,nan,1264,2,C:\coding\voki\data\SerieVertes1.pdf,1,[],0,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
055e0ce7a55ea957b1590c03a28e583a,"Anougba, Niangbô, N'da",einheimische Namen,nan,532,37,C:\coding\voki\data\SerieVertes4.pdf,4,[],0,NaN,NaN,False,NaN
0ffc3f0e5dd099af0a92ad2d1e91f20a,un sorcier/une sorcière,ein Zauberer/eine Zau-,Les Africains s'adressent quelquefois à ~ quan...,576,37,C:\coding\voki\data\SerieVertes4.pdf,4,[],0,NaN,NaN,False,NaN
ed59c85d1cbc1cf3e9b8dc4bf0fd42a1,un sous-préfet,ein Unterpräfekt,Le ~ est un représentant de l'Etat.,624,37,C:\coding\voki\data\SerieVertes4.pdf,4,[],0,NaN,NaN,False,NaN


In [32]:
df = trainer.db

In [33]:
df.columns

Index(['text', 'text_g', 'text_s', 'y_max', 'page', 'file', 'book', 'attempts',
       'streak', 'last_attempt_time', 'last_successful_attempt_time',
       'passed_vokibox', 'box_shortlisted'],
      dtype='object')

In [38]:
df[~df['last_attempt_time'].isna()].sort_values(by = 'last_attempt_time')

,text,text_g,text_s,y_max,page,file,book,attempts,streak,last_attempt_time,last_successful_attempt_time,passed_vokibox,box_shortlisted
hash,,,,,,,,,,,,,
52eea49610370c6968dbd549c2e5f6be,Paris,Hauptstadt Frankreichs,nan,1146,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Successful attempt at 2022-02-27 20:59:48: ""P...",3,2022-02-27 21:01:04,2022-02-27 21:01:04,True,NaN
2adc174acd08fa65f1df3b8f22abdb43,une leçon,eine Lektion,nan,943,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Failed attempt at 2022-02-27 20:59:20: ""une l...",3,2022-02-27 21:02:27,2022-02-27 21:02:27,True,NaN
54d9f89c97e39cbaf23893f701f06e0a,Arthur,männl. Vorname; hier: Name eines Papageis,nan,1079,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Failed attempt at 2022-02-27 20:59:31: ""arthu...",3,2022-02-27 21:02:32,2022-02-27 21:02:32,True,NaN
ce0efe3d25d6b36e8aba2ec454b48711,à,in,nan,1122,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Failed attempt at 2022-02-27 20:59:36: ""á"", S...",3,2022-02-27 21:02:39,2022-02-27 21:02:39,True,NaN
c0b716ccc9c4912796f42843354f270b,1 = un,eins,nan,965,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Successful attempt at 2022-02-27 20:59:25: ""1...",2,2022-02-27 21:03:36,2022-02-27 21:03:36,False,NaN
fc6f4b16c99f0cbcb3e6f18e169df236,Bonjour!,Guten Tag!,"~, Arthur!",1171,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Successful attempt at 2022-02-27 21:02:47: ""B...",2,2022-02-27 21:04:41,2022-02-27 21:04:41,False,NaN
2ced79c45d1ccdbdd2e01726d87e1dae,une rue,eine Straße,nan,1192,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Failed attempt at 2022-02-27 21:04:47: ""une ru""]",0,2022-02-27 21:04:47,NaN,False,NaN
66932ff7f2834ad68c9a9dbfa393d3ec,c'est,das ist,nan,1216,2,C:\coding\voki\data\SerieVertes1.pdf,1,"[Successful attempt at 2022-02-27 21:04:51: ""c...",1,2022-02-27 21:04:51,2022-02-27 21:04:51,False,NaN


In [12]:
from pyautogui import typewrite

print("enter folder name: ")
typewrite("Default Value");folder = input()

enter folder name: 


 asd


In [19]:
t2.db.loc[hash_,'attempts']

'[Successful attempt at 2022-02-27 15:08:51: "une leçon"]'

In [13]:
print(1)
print("\033[1ARandom string value")

1
Random string value


In [34]:
fname = trainer_dir / 'deleteme.pk'
a=[1]
with open(fname,'wb') as fh:
    pickle.dump(a,fh)


In [37]:
with open(fname,'rb') as fh:
    b=pickle.load(fh)

In [39]:
properties

{'DB_PATH': WindowsPath('C:/coding/voki/db/v1/db.csv'),
 'ROOT': WindowsPath('C:/coding/voki/db/v1'),
 'box': level 1
 level 2
 level 3
 level 4
 level 5
 level 6,
 'fpath': WindowsPath('C:/coding/voki/db/v1')}

In [6]:
hash_ = trainer.db.iloc[1].name
trainer.box.onboard_card(hash_,level=6)
trainer.show_box()

Voki Box with 1/20 cards
level 1
level 2
level 3
level 4
level 5
level 6
    eins


In [7]:
trainer.attempt_card_in_level(lvl=6)


eins 1 = un


correct!


True

In [8]:
trainer.show_box()

Voki Box with 0/20 cards
level 1
level 2
level 3
level 4
level 5
level 6


In [43]:
a=[1,2,3,4]
a.remove(1)
a

[2, 3, 4]

In [44]:
a.remove(1)

ValueError: list.remove(x): x not in list

In [17]:
import time
start = time.time()
trainer.save
print(f'{time.time()-start}')

0.0


In [31]:
trainer2 = vo.Trainer.load

eine Lektion un leçon


TypeError: string indices must be integers

In [30]:
trainer.show_box()

level 1
    eine Lektion
    eins
    männl. Vorname; hier: Name eines Papageis
    in
    Hauptstadt Frankreichs
    Guten Tag!
    eine Straße
    das ist
    Herr ...
    Familienname
    männl. Vorname
    mit
    und
    ein Papagei
    Arthur ist ein Papagei.
    Familienname
    ein Nachbar
    Abkürzung für Monsieur Lacombe
    Frau ...
    Guten Tag! (zu einer Frau)
level 2
level 3
level 4
level 5
level 6


In [29]:
trainer.box

level 1
    2adc174acd08fa65f1df3b8f22abdb43
    c0b716ccc9c4912796f42843354f270b
    54d9f89c97e39cbaf23893f701f06e0a
    ce0efe3d25d6b36e8aba2ec454b48711
    52eea49610370c6968dbd549c2e5f6be
    fc6f4b16c99f0cbcb3e6f18e169df236
    2ced79c45d1ccdbdd2e01726d87e1dae
    66932ff7f2834ad68c9a9dbfa393d3ec
    801dbee798c9ff4d905f7537f1fe0ded
    ad158daf1773b311a1a61929c9bf019b
    8f3a49dfcc49000a238bd8d22d791daf
    1476a2e343d5a537d9f8810549026068
    cc244c5a33ddab783d6b46d61ead9985
    fd27fc8176946666e0138595793f62e9
    082f11c3f3b61bfdaf48542e405ffc8d
    c2311b32c4ef2b824b2a36021e476eb1
    e9219b6871d2983193d85eba53b13ffb
    01af01828577f173672292a27647c435
    aa18060786f380cfd5413ea8143b150d
    04c8cf8e25668a9070abc170bf3a1c82
level 2
level 3
level 4
level 5
level 6

In [18]:
trainer.db.iat['2adc174acd08fa65f1df3b8f22abdb43',:]

ValueError: iAt based indexing can only have integer indexers

In [9]:
trainer.box.get_all_cards()

['2adc174acd08fa65f1df3b8f22abdb43',
 'c0b716ccc9c4912796f42843354f270b',
 '54d9f89c97e39cbaf23893f701f06e0a',
 'ce0efe3d25d6b36e8aba2ec454b48711',
 '52eea49610370c6968dbd549c2e5f6be',
 'fc6f4b16c99f0cbcb3e6f18e169df236',
 '2ced79c45d1ccdbdd2e01726d87e1dae',
 '66932ff7f2834ad68c9a9dbfa393d3ec',
 '801dbee798c9ff4d905f7537f1fe0ded',
 'ad158daf1773b311a1a61929c9bf019b',
 '8f3a49dfcc49000a238bd8d22d791daf',
 '1476a2e343d5a537d9f8810549026068',
 'cc244c5a33ddab783d6b46d61ead9985',
 'fd27fc8176946666e0138595793f62e9',
 '082f11c3f3b61bfdaf48542e405ffc8d',
 'c2311b32c4ef2b824b2a36021e476eb1',
 'e9219b6871d2983193d85eba53b13ffb',
 '01af01828577f173672292a27647c435',
 'aa18060786f380cfd5413ea8143b150d',
 '04c8cf8e25668a9070abc170bf3a1c82']

In [10]:
trainer.show_box()

level 1
    eine Lektion
    eins
    männl. Vorname; hier: Name eines Papageis
    in
    Hauptstadt Frankreichs
    Guten Tag!
    eine Straße
    das ist
    Herr ...
    Familienname
    männl. Vorname
    mit
    und
    ein Papagei
    Arthur ist ein Papagei.
    Familienname
    ein Nachbar
    Abkürzung für Monsieur Lacombe
    Frau ...
    Guten Tag! (zu einer Frau)
level 2
level 3
level 4
level 5
level 6


In [213]:
myvokibox.attempt_card_in_level(4,df);

überall partout


correct!
congrats! card "partout" made it through the vokibox!


In [189]:
myvokibox.attempt_card_in_level(0,df)

überall partout


correct!


In [170]:
myvokibox

level 0
    442165381189763710
level 1
level 2
level 3
level 4
level 5
    -4980411815809341467

In [165]:
myvokibox.attempt_card_in_level(1,df)

vokibox lvl 1 is empty. you need to fill it first


In [166]:
myvokibox.attempt_card_in_level(2,df)

überall partout


correct!


In [167]:
myvokibox.attempt_card_in_level(3,df)

überall partout


correct!


In [169]:
myvokibox.attempt_card_in_level(4,df)

überall partout


correct!


In [171]:
myvokibox.attempt_card_in_level(5,df)

überall partout


correct!
congrats! card partout made it through thevokibox!


In [172]:
myvokibox

level 0
    442165381189763710
level 1
level 2
level 3
level 4
level 5

In [140]:
class card:
    # a class for vocabulary cards

    def __init__(self, props):
        # the basic card that has typical properties of a vocabulary card
        self.french_phrase = props['french']['phrase']
        self.french_sentences =      props['french']['sentences']
        self.french_comments = props['french']['comments']
        self.german_phrase = props['german']['phrase']
        self.german_sentences = props['german']['sentences']
        self.german_comments = props['german']['comments']

        self.created = datetime.datetime.now().replace(microsecond=0)
        self.attempt_history = pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                             'result': pd.Series(dtype='bool'),
                                             'answer': pd.Series(dtype='str')})
        self.id = hash(self.french_phrase)
        self.streak = 0

    def attempt(self, guess):
        now = datetime.datetime.now().replace(microsecond=0)
        result = self.check_attempt(guess) #true/False

        self.attempt_history = self.attempt_history.append(pd.DataFrame({'time': [now], 
                                                                        'result': [result], 
                                                                        'answer': [guess]}))
        if result:
            self.streak += 1
        else:
            self.streak = np.floor(self.streak/2)

    def check_attempt(self,answer):
        if answer == self.french_phrase:
            return True
        text = input(f'type "yes" if you think they match:\n {answer} \n {self.french_phrase}\n')
        if text == 'yes':
            return True
        return False
    
    def __repr__(self):
        
        if len(self.attempt_history)>0:
            last_attempt = ''
        str_ = f"""voki card 
        {self.id}
        french: {self.french_phrase}
        german: {self.german_phrase} 
        last_attempt: """
        return str_
    
    


    @staticmethod
    def get_card_from_db(db,id):
        pass
    

        
    



        

        
def main(dbfile):
    #1 laod database
    #2 run program
    #3 save databas
    pass



In [185]:
mycard = card_from_id(4980411815809341467,df)

IndexError: single positional indexer is out-of-bounds

In [177]:
mycard

{'french_phrase': 'la rue',
 'french_sentences': ['Dans la rue du Sentier, la mode est partout.'],
 'french_comments': [],
 'german_phrase': 'Strasse',
 'german_sentences': [],
 'german_comments': [],
 'created': Timestamp('2021-10-31 21:20:50'),
 'attempt_history': Empty DataFrame
 Columns: [time, result, answer]
 Index: [],
 'streak': 0,
 'id': 442165381189763710}

In [179]:
pd.DataFrame({key: [value] for key,value in mycard.items()})

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,german_comments,created,attempt_history,streak,id
0,la rue,"[Dans la rue du Sentier, la mode est partout.]",[],Strasse,[],[],2021-10-31 21:20:50,"Empty DataFrame Columns: [time, result, answer...",0,442165381189763710


In [170]:
root = Path.cwd()
data_dir = root.parent / 'data'
voki_db_file = data_dir / 'voki_db.pkl'
box_db_file = data_dir / 'box_db.pkl'

# main()

# df_voki = pd.read_pickle(voki_db_file)
# voki_box = box_db_load(box_db_file)

In [161]:
voki_box

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [162]:
df_voki

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0


In [115]:
my_vbox.onboard_card(voki_card1.id)
my_vbox

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [121]:
box_db_load(box_db_file)

level 0
    5116320665103601376
    463857425755945451
level 1
level 2
level 3
level 4
level 5

In [3]:
card_1_dict = {'french_phrase': 'partout', 
                'french_sentences': ['Dans la rue du Sentier, la mode est partout.'],
                'french_comments':[],
                'german_phrase' :'überall',
                'german_sentences':[],
                'german_comments':[],
                'created': datetime.datetime.now().replace(microsecond=0),
                'attempt_history' : pd.DataFrame({'time': pd.Series(dtype='datetime64[ns]'),
                                             'result': pd.Series(dtype='bool'),
                                             'answer': pd.Series(dtype='str')})
                'streak': 0
                'id': hash('partout')
              }
                     
card_2_properties = {'french':{'phrase': 'la rue', 
                               'sentences': ['Julien habite dans la rue Daguerre.'],
                               'comments':[]},
                     'german':{'phrase':'Strasse',
                               'sentences':[],
                               'comments':[]},
                    }   

In [168]:
a=[[]]
a

[[]]

In [4]:
voki_card1=card(card_1_properties)
voki_card1

voki card 
        463857425755945451
        french: partout
        german: überall 
        last_attempt: 

In [5]:
voki_card2=card(card_2_properties)
voki_card2

voki card 
        5116320665103601376
        french: la rue
        german: Strasse 
        last_attempt: 

In [91]:
voki_card1.attempt('partout2'                  )


type "yes" if you think they match:
 partout2 
 partout no


In [92]:
voki_card1.attempt_history

,time,result,answer
0,2021-10-31 08:58:40,1.0,partout
0,2021-10-31 09:17:37,1.0,partout
0,2021-10-31 09:17:40,0.0,partout2


In [16]:
pd.DataFrame(voki_card1.to_df())

TypeError: to_df() got an unexpected keyword argument 'index'

In [19]:
df_list = [pd.DataFrame(voki_card1.to_df()),
           pd.DataFrame(voki_card2.to_df())]
df_voki = pd.concat(df_list)

In [32]:
df_voki.to_pickle(voki_db_file)

In [26]:
df_voki

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0


In [35]:
a=pd.read_pickle(voki_db_file)
a

,french_phrase,french_sentences,french_comments,german_phrase,german_sentences,created,attempt_history,id,streak
0,partout,"[Dans la rue du Sentier, la mode est partout.]",[],überall,[],2021-10-31 13:48:03,"Empty DataFrame Columns: [time, result, answer...",463857425755945451,0
0,la rue,[Julien habite dans la rue Daguerre.],[],Strasse,[],2021-10-31 13:48:04,"Empty DataFrame Columns: [time, result, answer...",5116320665103601376,0


In [ ]:
class Parent:
    def __init__(self,parent_id,member_id):
        self.parent_id = parent_id
        self.member = Member(member_id)

    def who_is_your_member(self):
        print(self.member)
        
class Member: # not inhereting from Parent
    def __init__(self, member_id):
        self.member_id = member_id
        
    def __repr__(self):
        return f'Member {self.member_id} of parent {self.parent_id}
        
    
    
myparent = Parent(1,2)
myparent.who_is_your_member()

>> Child 1 of parent 2

In [ ]:
class VocTrainer:
    def __init__(self,pandas_df):
        self.vocabulary_databas = pandas_df
        self.trainingbox = VocTrainingBox()
        self.trainingbox.fill(self.vocabulary_databas)
        
        
